In [2]:
import pandas as pd
from global_variables.global_variables import filename_proc, sleep_activities
from supporting_func.data_summaries import strip_activity, combine_sleep

In [ ]:
#download just the sleep data
df = pd.read_csv(filename_proc)
df = df[sleep_activities]
# print(df)

In [ ]:
from supporting_func.data_summaries import strip_activity, combine_sleep

#create pd start sleep, stop sleep, time slept, date
sleep_summary = strip_activity('Sleep', df)
print(sleep_summary)


In [ ]:
from global_variables.global_variables import start_date, end_date
daily_sleep_sum = combine_sleep(sleep_summary,start_date, end_date)
print(daily_sleep_sum)

In [ ]:
#find the distributions for time slept, average bed time, average wake time
average_sleep = daily_sleep_sum['total_sleep'].mean()
std_sleep = daily_sleep_sum['total_sleep'].std()

In [3]:
df = pd.read_csv('processed_data/hh111/hh111_preprocessed_60sw.csv')
df = df[sleep_activities]
sleep_summary = strip_activity('Sleep', df)
print(sleep_summary)

NameError: name 'strip_activity' is not defined